In [ ]:
!nvidia-smi

Fri Dec  5 13:36:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   51C    P8             13W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 39.9 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.57.2
Uninstalling transformers-4.57.2:
  Successfully uninstalled transformers-4.57.2
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 147.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 36.4 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd

# from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

# Download NLTK sentence tokenizer
nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from transformers import AutoTokenizer, PegasusForConditionalGeneration

model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

ARTICLE_TO_SUMMARIZE = (
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
)
inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"])
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


"California's largest electricity provider has turned off power to hundreds of thousands of customers."

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

## Fine Tuning

In [ ]:
model = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)



tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
!pip install datasets --quiet

In [ ]:
from datasets import load_dataset
import pandas as pd
import os


os.makedirs("summarizer-data", exist_ok=True)

#  Load CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# OPTIONAL: use smaller subset so it's lighter (uncomment if needed)
# dataset["train"] = dataset["train"].select(range(20000))
# dataset["validation"] = dataset["validation"].select(range(2000))
# dataset["test"] = dataset["test"].select(range(2000))

# 3) Convert to CSV and save inside summarizer-data folder
dataset["train"].to_pandas().to_csv("summarizer-data/train.csv", index=False)
dataset["validation"].to_pandas().to_csv("summarizer-data/val.csv", index=False)
dataset["test"].to_pandas().to_csv("summarizer-data/test.csv", index=False)

# 4) Zip the folder
!zip -r summarizer-data.zip summarizer-data


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

  adding: summarizer-data/ (stored 0%)
  adding: summarizer-data/train.csv (deflated 62%)
  adding: summarizer-data/test.csv (deflated 63%)
  adding: summarizer-data/val.csv (deflated 63%)


In [ ]:
from datasets import load_dataset
import pandas as pd
import os

# Create folder for smaller dataset
os.makedirs("summarizer-data", exist_ok=True)

# Load full dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")


train_small = dataset["train"].select(range(2000))        # 2000 samples
val_small = dataset["validation"].select(range(500))       # 500 samples
test_small = dataset["test"].select(range(500))            # 500 samples

# Convert to CSV
train_small.to_pandas().to_csv("summarizer-data/train.csv", index=False)
val_small.to_pandas().to_csv("summarizer-data/val.csv", index=False)
test_small.to_pandas().to_csv("summarizer-data/test.csv", index=False)

!zip -r summarizer-data.zip summarizer-data


updating: summarizer-data/ (stored 0%)
updating: summarizer-data/train.csv (deflated 62%)
updating: summarizer-data/test.csv (deflated 61%)
updating: summarizer-data/val.csv (deflated 60%)


In [ ]:
# Download & unzip data
!wget https://raw.githubusercontent.com/NaVeen913/Text-Summarization/main/summarizer-data.zip
!unzip -o summarizer-data.zip


--2025-12-05 15:28:58--  https://raw.githubusercontent.com/NaVeen913/Text-Summarization/main/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4475191 (4.3M) [application/zip]
Saving to: ‘summarizer-data.zip.1’

summarizer-data.zip 100%[===================>]   4.27M  --.-KB/s    in 0.01s   

2025-12-05 15:28:59 (310 MB/s) - ‘summarizer-data.zip.1’ saved [4475191/4475191]

Archive:  summarizer-data.zip
  inflating: summarizer-data/train.csv  
  inflating: summarizer-data/test.csv  
  inflating: summarizer-data/val.csv  


In [ ]:
import pandas as pd

train_df = pd.read_csv("summarizer-data/train.csv")
val_df = pd.read_csv("summarizer-data/val.csv")
test_df = pd.read_csv("summarizer-data/test.csv")

train_df.head()


,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def summarize(text):
    inputs = tokenizer(text, truncation=True, padding="longest", return_tensors="pt").to(model_pegasus.device)

    summary_ids = model_pegasus.generate(
        inputs["input_ids"],
        max_length=128,
        num_beams=5,
        early_stopping=True
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
sample_text = train_df['article'][0] if 'article' in train_df.columns else train_df.iloc[0,0]
summary = summarize(sample_text)

print("Original Text:\n", sample_text[:500], "...")
print("\nSummary:\n", summary)


Original Text:
 LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as s ...

Summary:
 Harry Potter star Daniel Radcliffe gains access to a reported £20 million fortune .<n>Young actor says he has no plans to fritter his cash away .<n>Radcliffe's earnings from the first five Potter films have been held in a trust fund .


In [ ]:
!pip install -q transformers datasets rouge_score sentencepiece


In [ ]:
import pandas as pd

train_df = pd.read_csv("summarizer-data/train.csv")
val_df   = pd.read_csv("summarizer-data/val.csv")
test_df  = pd.read_csv("summarizer-data/test.csv")

print(train_df.columns)
train_df.head()


Index(['article', 'highlights', 'id'], dtype='object')


,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


In [ ]:
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq
)
import torch

#  Model & tokenizer
model_name = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_pegasus = model_pegasus.to(device)

# Convert pandas -> HF Dataset
train_ds = Dataset.from_pandas(train_df)
val_ds   = Dataset.from_pandas(val_df)
test_ds  = Dataset.from_pandas(test_df)

#  Preprocessing / tokenization function
max_input_length = 512
max_target_length = 64

def preprocess_function(batch):

    inputs = batch["article"]
    targets = batch["highlights"]

    # tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True
    )

    # tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
train_tokenized = train_ds.map(preprocess_function, batched=True, remove_columns=train_ds.column_names)
val_tokenized   = val_ds.map(preprocess_function, batched=True, remove_columns=val_ds.column_names)
test_tokenized  = test_ds.map(preprocess_function, batched=True, remove_columns=test_ds.column_names)

train_tokenized[0]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

{'input_ids': [45933,
  108,
  2159,
  143,
  25445,
  158,
  1315,
  5849,
  10173,
  2187,
  4767,
  59988,
  6602,
  489,
  112,
  114,
  1668,
  25946,
  604,
  38589,
  17593,
  604,
  158,
  9490,
  130,
  178,
  3043,
  1204,
  124,
  1491,
  108,
  155,
  178,
  21031,
  109,
  408,
  576,
  131,
  144,
  2682,
  114,
  7779,
  124,
  342,
  107,
  4767,
  59988,
  130,
  5849,
  10173,
  115,
  198,
  30827,
  10173,
  111,
  109,
  4257,
  113,
  109,
  7045,
  194,
  413,
  109,
  11313,
  113,
  20581,
  22803,
  116,
  279,
  109,
  278,
  108,
  109,
  758,
  5102,
  649,
  178,
  148,
  220,
  1017,
  112,
  94670,
  169,
  1325,
  429,
  124,
  991,
  1732,
  108,
  2256,
  111,
  8913,
  1829,
  107,
  198,
  187,
  272,
  131,
  144,
  511,
  112,
  129,
  156,
  113,
  274,
  200,
  170,
  108,
  130,
  783,
  130,
  157,
  795,
  7739,
  5008,
  631,
  1118,
  114,
  2926,
  1537,
  439,
  949,
  132,
  364,
  984,
  745,
  178,
  898,
  142,
  2676,
  27348,
  1678

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_pegasus)


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

batch_size = 2
num_epochs = 2

training_args = Seq2SeqTrainingArguments(
    output_dir="pegasus-summarizer-checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=num_epochs,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_steps=50,
    report_to=[]
)


TypeError: Seq2SeqTrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
!pip install -U transformers datasets rouge_score sentencepiece -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 55.9 MB/s eta 0:00:00


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch

batch_size = 2
num_epochs = 2


training_args = Seq2SeqTrainingArguments(
    "pegasus-summarizer-checkpoints"
)


training_args.num_train_epochs = num_epochs
training_args.per_device_train_batch_size = batch_size
training_args.per_device_eval_batch_size = batch_size
training_args.learning_rate = 5e-5
training_args.weight_decay = 0.01
training_args.logging_steps = 50
training_args.save_total_limit = 2


if hasattr(training_args, "evaluation_strategy"):
    training_args.evaluation_strategy = "epoch"
if hasattr(training_args, "save_strategy"):
    training_args.save_strategy = "epoch"
if hasattr(training_args, "predict_with_generate"):
    training_args.predict_with_generate = True


if hasattr(training_args, "fp16"):
    training_args.fp16 = torch.cuda.is_available()



In [ ]:
!pip install -q evaluate rouge_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00


In [ ]:
import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]
    return preds, labels

def compute_metrics(eval_pred):
    predictions, labels = eval_pred


    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)


    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)


    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
    )

    result = {key: value * 100 for key, value in result.items()}


    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)


    return {k: round(v, 4) for k in result.items()}


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
import os
os.environ.pop("WANDB_DISABLED", None)


'true'

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch

batch_size = 2
num_epochs = 2

training_args = Seq2SeqTrainingArguments("pegasus-summarizer-checkpoints")

training_args.num_train_epochs = num_epochs
training_args.per_device_train_batch_size = batch_size
training_args.per_device_eval_batch_size = batch_size
training_args.learning_rate = 5e-5
training_args.weight_decay = 0.01
training_args.logging_steps = 50
training_args.save_total_limit = 2

if hasattr(training_args, "evaluation_strategy"):
    training_args.evaluation_strategy = "epoch"
if hasattr(training_args, "save_strategy"):
    training_args.save_strategy = "epoch"
if hasattr(training_args, "predict_with_generate"):
    training_args.predict_with_generate = True


if hasattr(training_args, "report_to"):
    training_args.report_to = []      # or ["none"]


In [ ]:
trainer = Seq2SeqTrainer(
    model=model_pegasus,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


/tmp/ipython-input-469812126.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
50,1.828500
100,1.333900
150,1.262700
200,1.237800
250,1.226400
300,1.274900
350,1.213600
400,1.202600
450,1.134200
500,1.156700


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=2000, training_loss=1.125216230392456, metrics={'train_runtime': 776.4259, 'train_samples_per_second': 5.152, 'train_steps_per_second': 2.576, 'total_flos': 5670963313360896.0, 'train_loss': 1.125216230392456, 'epoch': 2.0})

In [ ]:
del trainer


In [ ]:
import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]
    return preds, labels

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    # convert to percentage
    final_result = {}
    for key, value in result.items():
        final_result[key] = round(value * 100, 4)

    # add average generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    final_result["gen_len"] = round(np.mean(prediction_lens), 4)

    return final_result


In [ ]:
trainer = Seq2SeqTrainer(
    model=model_pegasus,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-2623389897.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
val_metrics = trainer.evaluate()
print(val_metrics)


{'eval_loss': 1.902452826499939, 'eval_model_preparation_time': 0.0081, 'eval_rouge1': 36.2411, 'eval_rouge2': 16.2443, 'eval_rougeL': 26.9911, 'eval_rougeLsum': 26.956, 'eval_gen_len': 58.136, 'eval_runtime': 523.8676, 'eval_samples_per_second': 0.954, 'eval_steps_per_second': 0.477}


In [ ]:
test_results = trainer.predict(test_tokenized)
print("Test metrics:\n", test_results.metrics)


Test metrics:
 {'test_loss': 1.8677458763122559, 'test_model_preparation_time': 0.0081, 'test_rouge1': 36.3839, 'test_rouge2': 16.0071, 'test_rougeL': 26.8343, 'test_rougeLsum': 26.7889, 'test_gen_len': 57.232, 'test_runtime': 518.9973, 'test_samples_per_second': 0.963, 'test_steps_per_second': 0.482}


In [ ]:
save_dir = "pegasus-text-summarizer"

trainer.save_model(save_dir)          # saves model weights
tokenizer.save_pretrained(save_dir)   # saves tokenizer

print("Model saved to:", save_dir)


Model saved to: pegasus-text-summarizer


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

loaded_tokenizer = AutoTokenizer.from_pretrained("pegasus-text-summarizer")
loaded_model = AutoModelForSeq2SeqLM.from_pretrained("pegasus-text-summarizer").to(device)

def generate_summary(text, max_len=128):
    inputs = loaded_tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    summary_ids = loaded_model.generate(
        inputs["input_ids"],
        num_beams=5,
        max_length=max_len,
        early_stopping=True
    )

    return loaded_tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
sample_text = train_df["article"][0]
print("ORIGINAL:\n", sample_text[:600], "...\n")
print("SUMMARY:\n", generate_summary(sample_text))


ORIGINAL:
 LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar,"  ...

SUMMARY:
 Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 on Monday . Young actor says he has no plans to fritter his cash away . Radcliffe's earnings from the first five Potter films have been held in a trust fund .
